In [1]:
# imports

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import anthropic
from IPython.display import Markdown, display, update_display
import requests

In [2]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")
anthropic_api_key = os.getenv("ANTHROPIC_API_KEY")
# google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")

if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

OpenAI API Key exists and begins sk-proj-
Anthropic API Key not set


In [3]:
openai = OpenAI()
claude = anthropic.Anthropic()

In [4]:
# original propmt
system_message = """
You are an assistant that is very good at determining what tool to use to solve queries.
"""

ai_programming = """

TOOLS

You have two tools:

1. A calculator tool that can perform basic arithmetic operations, such as addition, subtraction, multiplication, and division. If this tool is used then respond as detailed in the OUTPUT section with "next": "do_calculation"
2. A jokes tool that can provide a light-hearted joke for the audience reqested. If this tool is used then respond as detailed in <OUTPUT> with "next": "do_joke" and "audience": "audience requested"


OUTPUT

For a user query, determine the best tool. If none of the tools are appropriate, respond with "None".

EXAMPLES

    First Example
    
    <OUTPUT>
    {"tool": "calculator", "next": "do_calculation", "arguments": {"num1": 5, "num2": 5, "operation": "addition"}} 
    <OUTPUT>

    Second Example

    <OUTPUT>
    {"tool": "joke", "next": "do_joke", "audience": "Pythonistas" }} 
    <OUTPUT>
    
Thank you for your help.
"""

system_message += ai_programming

In [5]:
user_prompt = "What is 5 times 9?"
user_prompt = "Tell me a joke when I am doing stand up at a Builders Conference"

# user_prompt = "Tell me a joke when I am doing stand up at a Builders Conference"

prompts = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_prompt},
]

# GPT-3.5-Turbo

response = openai.chat.completions.create(model="gpt-3.5-turbo", messages=prompts)

print(response.choices[0].message.content)

output = response.choices[0].message.content.replace("\n", "")

<OUTPUT>
{"tool": "joke", "next": "do_joke", "audience": "Builders Conference"}
<OUTPUT>


In [6]:
json_output = output.replace("<OUTPUT>", "").strip()
json_output = json.loads(json_output)
print(json_output)
do_next = json_output["next"]

{'tool': 'joke', 'next': 'do_joke', 'audience': 'Builders Conference'}


In [ ]:
if do_next == "do_calculation":
    tool = json_output["tool"]
    num1 = json_output["arguments"]["num1"]
    num2 = json_output["arguments"]["num2"]
    operation = json_output["arguments"]["operation"]
    if operation == "addition":
        answer = num1 + num2
    elif operation == "subtraction":
        answer = num1 - num2
    elif operation == "multiplication":
        answer = num1 * num2
    elif operation == "division":
        answer = num1 / num2
    print(f"{num1} {operation} {num2} = {answer}")

In [8]:
if do_next == "do_joke":
    get_random_joke_internet = requests.get(
        "https://official-joke-api.appspot.com/random_joke"
    )

    print(get_random_joke_internet.json())

{'type': 'programming', 'setup': 'Hey, wanna hear a joke?', 'punchline': 'Parsing HTML with regex.', 'id': 395}
